<a href="https://colab.research.google.com/github/chouhandiksha/bigdataproject/blob/main/notebooks/Analyse%20NY%20Compare%20Part%20Time%20Work%20Percentage%20For%20Wealthiest%20And%20Poorest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyse NY Compare Part Time Work Percentage For Wealthiest And Poorest

*Instructions:*

1. Execute the first code cell.
2. There will be a link to follow in order to authorize the google account for drive. Go to that link.
3. A code to authorize the google account will be generated. Copy the code generated.
4. Go back to the cell where the process of mounting the drive is running. Paste the generated code from step 3 to the text box in the cell and press enter.

**Spark SQL Documentation:** 
https://spark.apache.org/docs/2.2.0/sql-programming-guide.html

In [ ]:
# add time information at the end of every cell
!pip install ipython-autotime
%load_ext autotime

time: 3.1 ms (started: 2021-04-22 19:26:31 +00:00)


In [ ]:
# Install required dependancies
!pip install pyspark
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 40.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=c5331bf142dc76af2c04c1f78d00476dc9a43fc9f35376367c428e193f05ec9e
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 36.5 MB of archives.
After this 

In [ ]:
# Import modules
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from pathlib import Path
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

time: 105 ms (started: 2021-04-22 19:27:23 +00:00)


In [ ]:
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

time: 290 ms (started: 2021-04-22 19:27:23 +00:00)


In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

time: 4.89 s (started: 2021-04-22 19:27:23 +00:00)


In [ ]:
#sc.stop()

time: 1.49 ms (started: 2021-04-22 19:27:28 +00:00)


In [ ]:
# Mount drive with data
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 27.7 s (started: 2021-04-22 19:27:28 +00:00)


In [ ]:
# Set path to data folder
path = Path('drive/MyDrive/big-data-project/data/clean-data')
city = 'ny'

time: 1.77 ms (started: 2021-04-22 19:27:56 +00:00)


In [ ]:
# Read data into dataframe
df_soc = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'social/2020/*.csv'))
df_soc.show()

+---+------------+--------------------+--------------------+------------+---------------------------+--------------------------+------------------------------------------+----------------------------+----------------------+------------------------+--------------------+-------------------------------+-------------------------------+--------------------+-------------------------+--------------------------+----------------------+----------------------------+---------------------------+-----------------------------+
|_c0|         cbg|    date_range_start|      date_range_end|device_count|distance_traveled_from_home|bucketed_distance_traveled|median_dwell_at_bucketed_distance_traveled|completely_home_device_count|median_home_dwell_time|bucketed_home_dwell_time|at_home_by_each_hour|part_time_work_behavior_devices|full_time_work_behavior_devices|    destination_cbgs|delivery_behavior_devices|median_non_home_dwell_time|candidate_device_count|bucketed_away_from_home_time|median_percentage_time_

In [ ]:
df_soc.createOrReplaceTempView('clean_ny')
df_soc = spark.sql('SELECT cbg, date_range_start as date, device_count, part_time_work_behavior_devices FROM clean_ny WHERE device_count > 5')
df_soc.show()

+------------+--------------------+------------+-------------------------------+
|         cbg|                date|device_count|part_time_work_behavior_devices|
+------------+--------------------+------------+-------------------------------+
|360470064002|2020-02-03T00:00:...|          64|                              7|
|360810384001|2020-02-03T00:00:...|          84|                             12|
|360850170103|2020-02-03T00:00:...|         250|                             41|
|360050213021|2020-02-03T00:00:...|          92|                             11|
|360050253004|2020-02-03T00:00:...|          52|                              7|
|360470385003|2020-02-03T00:00:...|          37|                              4|
|360470406002|2020-02-03T00:00:...|          46|                              7|
|360470428003|2020-02-03T00:00:...|          84|                              2|
|360471130002|2020-02-03T00:00:...|          84|                             13|
|360811029001|2020-02-03T00:

In [ ]:
df_soc = df_soc.withColumn("date",
    df_soc['date'].substr(0, 10))

time: 46 ms (started: 2021-04-22 19:28:28 +00:00)


In [ ]:
df_soc.show()

+------------+----------+------------+-------------------------------+
|         cbg|      date|device_count|part_time_work_behavior_devices|
+------------+----------+------------+-------------------------------+
|360470064002|2020-02-03|          64|                              7|
|360810384001|2020-02-03|          84|                             12|
|360850170103|2020-02-03|         250|                             41|
|360050213021|2020-02-03|          92|                             11|
|360050253004|2020-02-03|          52|                              7|
|360470385003|2020-02-03|          37|                              4|
|360470406002|2020-02-03|          46|                              7|
|360470428003|2020-02-03|          84|                              2|
|360471130002|2020-02-03|          84|                             13|
|360811029001|2020-02-03|         256|                             24|
|360050177022|2020-02-03|          53|                              2|
|36005

In [ ]:
# Add completely home percentage column
# df_soc = df_soc.withColumn('completely_home_percentage', (df_soc['completely_home_device_count']/df_soc['device_count']) * 100)
df_soc = df_soc.withColumn('part_time_work_percentage', (df_soc['part_time_work_behavior_devices']/df_soc['device_count']) * 100)
# df_soc = df_soc.withColumn('full_time_work_percentage', (df_soc['full_time_work_behavior_devices']/df_soc['device_count']) * 100)

#df_soc.show()

time: 69.3 ms (started: 2021-04-22 19:28:28 +00:00)


In [ ]:
#Create temp view
df_soc.createOrReplaceTempView('mobility')

time: 28 ms (started: 2021-04-22 19:28:28 +00:00)




---


**Demographic data**





---



In [ ]:
# Read poverty data
# Read data into RDD
df_demographic = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/'ny.csv'))
df_demographic.createOrReplaceTempView('demographic')
df_demographic.show()

+------------+---------+------------------+------------------+
|         cbg|pop_total|poverty_percentage|    perc_whiteonly|
+------------+---------+------------------+------------------+
|360050175002|     1110| 55.22522522522523|14.774774774774773|
|360050141001|     1445| 36.74740484429066| 19.16955017301038|
|360050145001|     2003| 39.47667804323095|  8.53719420868697|
|360050075002|     2731|22.812156719150494|4.5404613694617355|
|360050418001|      936|18.859649122807017| 9.081196581196581|
|360050125001|     2935| 43.24693042291951|12.742759795570699|
|360050127012|     1061| 43.73232799245994|18.378887841658813|
|360050054003|     1031| 57.12900096993211| 28.12803103782735|
|360050027011|     1364|  65.9090909090909|14.589442815249267|
|360050458001|     1168| 37.15753424657534| 2.910958904109589|
|360050224011|     1291|19.907048799380327|27.420604182804027|
|360050389001|     1143| 53.94265232974911|30.708661417322837|
|360050071002|     1661|           37.4375|11.499096929

In [ ]:
df_mob_demo = spark.sql('SELECT mobility.*, demographic.poverty_percentage from mobility INNER JOIN demographic ON mobility.cbg = demographic.cbg')
df_mob_demo.createOrReplaceTempView('demographic_mobility')
df_mob_demo.show()


+------------+----------+------------+-------------------------------+-------------------------+------------------+
|         cbg|      date|device_count|part_time_work_behavior_devices|part_time_work_percentage|poverty_percentage|
+------------+----------+------------+-------------------------------+-------------------------+------------------+
|360470064002|2020-02-03|          64|                              7|                  10.9375| 19.22705314009662|
|360810384001|2020-02-03|          84|                             12|       14.285714285714285|12.291350531107739|
|360850170103|2020-02-03|         250|                             41|       16.400000000000002|13.652802893309223|
|360050213021|2020-02-03|          92|                             11|       11.956521739130435| 43.07328605200946|
|360050253004|2020-02-03|          52|                              7|       13.461538461538462|24.824120603015075|
|360470385003|2020-02-03|          37|                              4|  

In [ ]:
# DIVIDING THE PEOPLE TOP 20 PERCENT AND BOTTOM 20 PERCENT

df_mob_demo_high_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage < 20')
grouped_df_mob_demo_high_class = df_mob_demo_high_class.groupBy("date").mean("part_time_work_percentage").withColumnRenamed('avg(part_time_work_percentage)','part_time_work_percentage')
grouped_df_mob_demo_high_class.createOrReplaceTempView('mob_demo_high_class')


df_mob_demo_low_class =  spark.sql('SELECT * FROM demographic_mobility WHERE poverty_percentage > 80')
grouped_df_mob_demo_low_class = df_mob_demo_low_class.groupBy("date").mean("part_time_work_percentage").withColumnRenamed('avg(part_time_work_percentage)','part_time_work_percentage')
grouped_df_mob_demo_low_class.createOrReplaceTempView('mob_demo_low_class')

grouped_df_mob_demo_combined = spark.sql("""

SELECT h.date,h.part_time_work_percentage as top20_perc,
l.part_time_work_percentage as bottom20_perc  
FROM mob_demo_high_class h INNER JOIN mob_demo_low_class l ON h.date = l.date

""")


time: 359 ms (started: 2021-04-22 19:28:30 +00:00)


**Load 2019 stats**

In [ ]:
filename = '{}_social_summary_2019.csv'.format(city)
stat_2019 = spark.read.format('csv').option('header','true').option('quote',"\"").option('escape',"\"").load(str(path/city/filename))
stat_2019.show()
mean_2019 = stat_2019.collect()[1][6]


+-----+------------------+------------------+-----------------+------------------+--------------------------+-------------------------+-------------------------+--------------------+--------------------+--------------------+-----------------+
|  _c0|             total|   completely_home|   part_time_work|    full_time_work|percentage_completely_home|percentage_part_time_work|percentage_full_time_work|norm_completely_home| norm_part_time_work| norm_full_time_work|            month|
+-----+------------------+------------------+-----------------+------------------+--------------------------+-------------------------+-------------------------+--------------------+--------------------+--------------------+-----------------+
|count|         2290248.0|         2290248.0|        2290248.0|         2290248.0|                 2290248.0|                2290248.0|                2290248.0|           2290248.0|           2290248.0|           2290248.0|        2290248.0|
| mean| 80.34526610218632|28

In [ ]:
# calculate deviation form mean
grouped_df_mob_demo_combined= grouped_df_mob_demo_combined.withColumn("top20_deviation",
    grouped_df_mob_demo_combined['top20_perc'] - mean_2019)


time: 35.8 ms (started: 2021-04-22 19:28:32 +00:00)


In [ ]:
grouped_df_mob_demo_combined = grouped_df_mob_demo_combined.withColumn("bottom20_deviation",
    grouped_df_mob_demo_combined['bottom20_perc'] - mean_2019)

time: 23.5 ms (started: 2021-04-22 19:28:32 +00:00)



**Visualizations** 



In [ ]:
# create altair library theme 

def my_theme():
  return {
    'config': {
      'view': {'continuousHeight': 300, 'continuousWidth': 400},  # from the default theme
      'range': {'category': {'scheme': 'redblue'}}##['#FF4500','#a0aab4','#4E79A7']}}
    }
  }
alt.themes.register('my_theme', my_theme)
alt.themes.enable('my_theme')
# 

ThemeRegistry.enable('my_theme')

time: 14.6 ms (started: 2021-04-22 19:28:32 +00:00)


In [ ]:
def create_chart(df) :
  chart = alt.Chart(df).transform_fold(
    fold=['Bottom 20 % Deviation', 'Top 20 % Deviation'],
    as_ = ['Poverty %','part_time_value']
    
    ).mark_line().encode(
      x= alt.X('date:T', title = 'Date'),
      y= alt.Y('part_time_value:Q', title = 'Part Time Work %'),
      color='Poverty %:N'
    ).properties(width = 1000).interactive()

  rule = alt.Chart(pd.DataFrame({'Mean 2019':[0]})).mark_rule(color='#757575',strokeDash=[5,3], size=2).encode(
    y='Mean 2019',
    size=alt.value(2),
    )
  
  text = alt.Chart({'values':[{ 'y': 1}]}).mark_text(
    text='Mean 2019', angle=0
    ).encode(
      y=alt.Y('y:Q'),opacity=alt.value(0.4)
    )

  return  chart + rule + text


time: 36.3 ms (started: 2021-04-22 19:28:32 +00:00)


In [ ]:
def apply_date_filter(df,start_date,end_date) :
  mask = (df['date'] > start_date) & (df['date'] <= end_date)
  dfX = df.loc[mask]
  return dfX

time: 2.74 ms (started: 2021-04-22 19:28:32 +00:00)


In [ ]:
# create pandas dataframe for visualizations
df = grouped_df_mob_demo_combined.toPandas()
df

,date,top20_perc,bottom20_perc,top20_deviation,bottom20_deviation
0,2020-02-26,12.808218,6.311963,4.620591,-1.875664
1,2020-04-13,4.067297,4.518371,-4.120330,-3.669256
2,2020-06-24,6.134412,7.679739,-2.053215,-0.507889
3,2020-06-08,5.045353,5.097403,-3.142274,-3.090225
4,2020-09-12,6.224557,5.516194,-1.963070,-2.671433
...,...,...,...,...,...
361,2020-04-05,3.771187,3.306452,-4.416440,-4.881176
362,2020-05-01,4.023622,7.117794,-4.164006,-1.069833
363,2020-10-25,4.873666,4.195402,-3.313961,-3.992225
364,2020-12-28,6.751881,4.846154,-1.435746,-3.341473


time: 3min 23s (started: 2021-04-22 19:28:32 +00:00)


In [ ]:
df = df.rename(columns={'top20_perc': 'Top 20 %', 'bottom20_perc': 'Bottom 20 %','top20_deviation': 'Top 20 % Deviation', 'bottom20_deviation': 'Bottom 20 % Deviation'})

time: 3.26 ms (started: 2021-04-22 19:31:55 +00:00)


In [ ]:
# analysis for the entire year on monthly basis
df_str = df.copy(deep=True) #make a deep copy

df_str['date'] = pd.to_datetime(df_str['date'],utc= True) # extract the months

df_str = df_str.groupby([df_str['date']]).mean().rolling(10).mean()[0:] #find the mean for the days and roll up for every 10 days

chart = alt.Chart(df_str.reset_index()).transform_fold(
    fold=['Bottom 20 %', 'Top 20 %'],
    as_ = ['Poverty %','part_time_value']
    
    ).mark_line().encode(
      x='date:T',
      y= alt.Y('part_time_value:Q', title = 'Part Time %'), color='Poverty %:N'
    ).properties(width = 1000)
chart

alt.Chart(...)

time: 117 ms (started: 2021-04-22 19:31:55 +00:00)


In [ ]:
#Conversion of datetime format
df['date'] = pd.to_datetime(df['date'],utc= True)
#df['date'] = [datetime.datetime.date(d) for d in df['date']] 

time: 6.74 ms (started: 2021-04-22 19:31:55 +00:00)


Filtering based on important dates

**March 8, 2020:**
NYC issues guidelines to avoid densely packed buses, subways, or trains.

**March 12, 2020:**
Broadway shuts down.

**March 14, 2020:**
First two COVID-19 deaths in NYS.

**March 22, 2020:**
NYS on Pause Program begins, all non-essential workers must stay home.

**March 28, 2020:**
Governor Cuomo halts all nonessential construction sites in NYS.











In [ ]:
start_date = '2020-3-5'
end_date = '2020-3-31'
df1 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df1)
chart

alt.LayerChart(...)

time: 79.6 ms (started: 2021-04-22 19:31:55 +00:00)




**April 10, 2020:**
NYS records more COVID19 cases than any other country 









In [ ]:
start_date = '2020-4-2'
end_date = '2020-4-24'
df2 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df2)
chart
#df2

alt.LayerChart(...)

time: 59.4 ms (started: 2021-04-22 19:31:55 +00:00)



**July 6, 2020:** 
NYC begins Phase 3 of reopening, without indoor dining





In [ ]:
start_date = '2020-7-1'
end_date = '2020-7-16'
df3 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df3)
chart

alt.LayerChart(...)

time: 60.4 ms (started: 2021-04-22 19:31:55 +00:00)


**October 28, 2020:** New York tops a half-million confirmed COVID-19 cases


In [ ]:
start_date = '2020-10-20'
end_date = '2020-11-8'
df4 = apply_date_filter(df,start_date,end_date)
chart = create_chart(df4)
chart

alt.LayerChart(...)

time: 61.2 ms (started: 2021-04-22 19:31:55 +00:00)
